# Workflow Example 1

This is based on a simple workflow, where only one cube is chosen with one dimension to plot over and a restricted number of customization options.

This is not a typical workflow, but it's a good start.

In [1]:
import ipywidgets
import IPython.display
import iris

import glob
import numpy as np
import iris.quickplot as iplt
import matplotlib.pyplot as plt

/data/local/cbosley/mc/c/envs/cubebrowser/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [2]:
%matplotlib inline

# 1. Choose and load your cube

Create cube picker consiting of a text acceptor for the file path and a multiple selection pane for the cube selector

Please enter the path of your cube(s) and then select the cube or cubelist you would like to plot with.

In [3]:
path = ipywidgets.Text(
    description='Filepath',
    value='/tmp',
)
IPython.display.display(path)

In [4]:
options = glob.glob('{}/*'.format(path.value)) # Can we do this in a different form, so that we can see the options 
                                               # without the file path?

files = ipywidgets.SelectMultiple(
    description='Cube(list)',
    options=options
)
IPython.display.display(files)

At this point we need to see the cube or cubelist in order to make a decision about what to do next.  For example, if we have one cube then we can move on to the next stage (choosing the plot type) whereas if we have a list of cubes then we have to choose one of those cubes to carry forward and choose the plot type for.  Then, of course, there is the possibility that a user would want to plot several cubes with the same types and settings, so maybe this stage also needs a multiple cube selector.

This would be much easier with a file browser or some equivalent, which would allow the user to see what they are selecting (i.e. whether it is a folder or a file) and then have more control over the cubes they are selecting.  Then later on in the workflow, an option to plot another cube could follow.  This, however, would not allow for multiple plots and plot types with a single associated slider.

Hmmm......

In [86]:
cubes = files.value

for i in range(len(cubes)):
    print cubes[i]
    if cubes[i][-2:] == 'nc' or cubes[i][-2:] == 'pp': # This is currently not taking grib files into 
                                                           # account coz they are hard to identify.  Also no ff.
        cubelist = iris.load_raw(cubes[i])
    else:
        print 'This is not a cube, please try another selection.'

print cubelist        

../../iris-sample-data/sample_data/hybrid_height.nc
0: air_potential_temperature / (K)     (model_level_number: 15; grid_latitude: 100; grid_longitude: 100)


Select which cubes you would like to use if you have loaded a cubelist

In [87]:
to_plot = [cube.standard_name for cube in cubelist]

plots = ipywidgets.SelectMultiple(
description='Choose cube(s)',
options=to_plot
)

IPython.display.display(plots)

I feel that at this point we need a 'continue' button.  In actual fact we probably need several of these at various points in the workflow.  I am coming round to this way of thinking because at nearly every selection point, a dependency is created, so before the next set of options can be displayed, the current selection must be set in order to generate those options.

If this was not the case, I can't see how this is going to work.

You could offer only options which are applicable to every case, which is incredibly limited, or you could offer every single option available in every case, which would chuck out an error in many cases.  Neither of these feel ideal.

# 2. Choose your plot options

Create plot-type picker

In [88]:
plot_type_dict = {'contour': iplt.contour, 'contourf': iplt.contourf, 'pcolor': iplt.pcolor, 'outline': iplt.outline,
                  'pcolormesh': iplt.pcolormesh, 'plot': iplt.plot, 'points': iplt.points}

plot_types = plot_type_dict.keys()
plot_types.sort()

type = ipywidgets.Dropdown(
    options=plot_types,
    value='contour',
    description='Plot-type:')

IPython.display.display(type)

You will be able to use this later using:
```python
callable = plot_type_dict[type.value]
callable(plots.value)
```

Create axis coordinate pickers for x and y axis

In [89]:
base_cube = iris.load_cube(files.value, plots.value[0])
# base_cube is the first cube chosen from the list.
# This method will work provided that all the cubes chosen have the same coordinates.

coordinates = [(coord.name()) for coord in base_cube.coords()]
# This bit adds the cube phenomenon as an axis, so you could plot, for example, surface temp against time.
# It may be causing a problem though, albeit a very solvable one.
for i in range(len(plots.value)):
    coordinates.append(plots.value[i])

dim_x = ipywidgets.RadioButtons(
    description='Dimension for x:',
    options=coordinates)

IPython.display.display(dim_x)

In [90]:
if dim_x.value in coordinates:
    coordinates.remove(dim_x.value)

dim_y = ipywidgets.RadioButtons(
    description='Dimension for y:',
    options=coordinates)

IPython.display.display(dim_y)

In [94]:
if dim_x.value in coordinates:
    coordinates.remove(dim_x.value)
if dim_y.value in coordinates:
    coordinates.remove(dim_y.value)
for i in range(len(plots.value)):
    if plots.value[i] in coordinates:
        coordinates.remove(plots.value[i])
    
sliders = []
for i in range(len(coordinates)):
    slider = ipywidgets.IntSlider(
        value=0,
        min=0,
        max=len(base_cube.coord(coordinates[i]).points-1),
        step=1,
        description='Index of ' + coordinates[i])
    sliders.append(slider)
    IPython.display.display(sliders[i])


# 3. Choose your formatting options

Here we need to take stock of which type of plot has been chosen, and offer an appropriate set of formatting options for that plot type.  This could take a bit of work to be complete and comprehensive.

A big problem that I am having is that some options can be either a boolean or a set value, or a string, or a float, etc., and the list of options is really too long to offer as a dropdown.  It might be possible to add a string entry widget to the options of a drowdown widget, but this is going to make the code very complicated; probably unnecessarily so.  This means that the input options are going to be very restricted at this stage.

In [92]:
# All:
colors = ipywidgets.Dropdown(
    options=['None', 'blue', 'green', 'red', 'cyan', 'magenta', 'yellow', 'black', 'white'],
    value='blue',
    description='colors')
cmap = ipywidgets.Dropdown(
    options=['None', 'viridis', 'inferno', 'plasma', 'magma', 'jet', 'summer', 'autumn'],
    value='None',
    description='cmap')
norm = ipywidgets.Dropdown(
    options=['None', 'Autoscale(A)', 'Autoscale_None(A)', 'scaled()'],
    value='None',
    description='norm')
levels = ipywidgets.Text(
    description='levels',
    value='np.linspace(np.min(cube.data), np.max(cube.data), 10)')
origin = ipywidgets.Dropdown(
    options=['None', 'upper', 'lower', 'image'],
    value='None',
    description='origin')
extend = ipywidgets.Dropdown(
    options=['neither', 'both', 'min', 'max'],
    value='both',
    description='extend')
# Contour-only:
linestyles = ipywidgets.Dropdown(
    options=['None', 'solid', 'dashed', 'dashdot', 'dotted'],
    value='None',
    description='linestyle')
# Plot only:
label = ipywidgets.Text(
    description='label',
    value='Line Plot')
# Plot and Points only:
color = ipywidgets.Dropdown(
    options=['blue', 'green', 'red', 'cyan', 'magenta', 'yellow', 'black', 'white'],
    value='blue',
    description='color')
marker = ipywidgets.Dropdown(
    options=['-', '--', '-.', ':', '.', ',', 'o', 'v', '^', '<', '>', '*', 'x'],
    value='-',
    description='marker')
    
formatting = [colors, cmap, norm, levels, origin, extend]
    
if type.value in ['contourf', 'outline', 'pcolor', 'pcolormesh']:
    for i in range(len(formatting)):
        IPython.display.display(formatting[i])
elif type.value == 'contour':
    formatting.append(linestyles)
    for i in range(len(formatting)):
        IPython.display.display(formatting[i])
elif type.value == 'plot':
    formatting.append(label)
    formatting.append(color)
    formatting.append(marker)
    for i in range(len(formatting)):
        IPython.display.display(formatting[i])
elif type.value == 'points':
    formatting.remove(colors)
    formatting.append(color)
    formatting.append(marker)    
    for i in range(len(formatting)):
        IPython.display.display(formatting[i])
    

# 4. Make your plot

This will require a button which does all the things that make a plot when you press it.

In [110]:

callable = plot_type_dict[type.value]

plot_opts = {'colors':colors.value, 'cmap':cmap.value, 
                 'norm':norm.value, 'levels':eval(levels.value), 'origin':origin.value, 'extend':extend.value}
if type.value == 'contour':
    plot_opts['linestyles'] = linestyles.value
elif type.value == 'plot':
    plot_opts['label'] = label.value
    plot_opts['color'] = color.value
    plot_opts['marker'] = marker.value   
elif type.value == 'points':
    del plot_opts['colors']
    plot_opts['color'] = color.value
    plot_opts['marker'] = marker.value    

for j in range(len(plots.value)):
    cube = iris.load_cube(files.value, plots.value[j])
    # Now need to work out how to put dictionary values in cube index list
    callable(cube[], **plot_opts)
    iplt.show()

SyntaxError: invalid syntax (<ipython-input-110-1f4f194ba28e>, line 37)

In [119]:
dim_dict = {}
dim_list = []
# Find dimension coordinates which are not axis coordinates, and append them to a list
# For dimensions which are axis coordinates, put ':' in dictionary
for coord in ([(coord.name()) for coord in cube.coords()]):
    if cube.coord(coord) in cube.coords(dim_coords=True):
        if coord not in [dim_x.value, dim_y.value]:
            dim_list.append(coord)
        else:
            dim_dict[dim_x.value] = ':'
            dim_dict[dim_y.value] = ':'

# Extract index value of non-axis coordinates from slider list and add to dictionary
for i in range(len(coordinates)):
    if coordinates[i] in dim_list:
        dim_dict[dim_list[i]] = sliders[i].value

